# Load the cluster data from a shape file

In [1]:
import geopandas as gpd
import polars as pl

gdf = gpd.read_file("../data/Property_information.shp")

pdf = pl.from_pandas(gdf.astype({"geometry": "str"}))

cluster_data = pdf
cluster_data

BESTANDS_I,Forest_own,geometry
i64,i64,str
25037,56,"""POLYGON ((607358.363169 659658…"
25039,56,"""POLYGON ((607484.284218 659661…"
25036,56,"""POLYGON ((607349.684448 659662…"
24985,55,"""POLYGON ((607634.79149 6596605…"
25038,56,"""POLYGON ((607420.071156 659656…"
…,…,…
25323,224,"""POLYGON ((608431.419375 660024…"
61145,222,"""POLYGON ((607627.026502 659906…"
25469,114,"""POLYGON ((607915.91287 6600687…"


# Load regime data

In [16]:
pdf = pl.read_csv("../data/Vaaler_data.csv",
                  separator=';',
                  has_header=True,
                  infer_schema_length=500000)
pdf = pdf[[s.name for s in pdf if not (s.null_count() > 0)]]  # drop null columns
regime_data = pdf

In [17]:
regime_data

id,branch,year,area,age,reg_age,reg_n,reg_n_S,reg_n_P,reg_n_B,reg_hDom,reg_hDom_S,reg_hDom_P,reg_hDom_B,reg_d,Tr,Rg,INC,COST,Net,Harvested_V,Harvested_V_S,Harvested_V_P,Harvested_V_B,Harvested_V_log,Harvested_V_log_S,Harvested_V_log_P,Harvested_V_log_B,Harvested_V_pulp,Harvested_V_pulp_S,Harvested_V_pulp_P,Harvested_V_pulp_B,n,n_S,n_P,n_B,h,…,vol_mort_pine.1,vol_mort_birch.1,h_mort.1,h_mort_spruce.1,h_mort_pine.1,h_mort_birch.1,d_mort.1,d_mort_spruce.1,d_mort_pine.1,d_mort_birch.1,ba_mort.1,ba_mort_spruce.1,ba_mort_pine.1,ba_mort_birch.1,biomass,soil,products,biomass.1,soil.1,products.1,NPV,Decay_class_1_volume,Decay_class_2_volume,Decay_class_3_volume,Decay_class_4_volume,Decay_class_5_volume,d_mort_pine_y,d_mort_spruce_y,vol_mort_pine_y,vol_mort_spruce_y,year.1,TTWP,CAP,HAZ,LSWP,SFS,LTT
i64,i64,i64,f64,i64,i64,i64,f64,f64,i64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,f64,f64,f64,f64
24473,1,1,2.03555,116,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,307.0,0.0,142.0,165.0,21.1219,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0.877308,152.553,0.0,0.0,154.544,7.85294,0.0,43885.5,13.997524,0.0,0.0,0.0,0.0,43.6782,0.0,6.51497,0.0,1,0.791623,0,0.0,0.539321,0.0,0.84365
25533,1,1,2.45563,0,1,170,0.0,170.0,0,1.35618,0.0,1.35618,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,777.112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0,0.0,0.0,0.0,0.0
25533,1,2,2.45563,0,6,170,0.0,170.0,0,2.6482,0.0,2.6482,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,777.112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0,0.0,0.0,0.0,0.0
24473,1,2,2.03555,121,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,296.106,0.0,136.961,159.145,21.5568,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0.887889,154.544,7.85294,0.0,156.181,12.7199,0.0,43885.5,27.918172,0.0,0.0,0.0,0.0,88.42,0.0,13.22017,0.0,2,0.802918,0,0.0,0.565131,0.0,0.843649
25533,1,3,2.45563,0,11,170,0.0,170.0,0,4.0775,0.0,4.0775,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,777.112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
61166,5,10,0.389153,61,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,19,63,142393.0,45009.0,97384.0,280.287,140.143,140.143,0.0,135.421,135.421,135.421,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,98.2422,10.7366,0.0,0.0,26.0607,0.0,31725.6,14.841899,4.797826,3.260281,3.281431,1.12436,194.1876,194.1876,18.91532,18.91532,10,0.01143,0,0.0,0.015026,0.0,0.0
61166,6,10,0.389153,0,5,3800,1910.0,90.0,1800,0.65,0.65,0.65,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,23.4304,0.0,0.0,15.8551,0.0,29261.1,6.522975,4.797826,3.260281,3.281431,1.12436,163.3861,163.3861,14.57784,14.57784,10,0.0,0,0.0,0.0,0.0,0.0
61166,7,10,0.389153,0,0,5200,1600.0,100.0,3500,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,23.4304,0.0,0.0,15.8551,0.0,32535.6,6.522975,4.797826,3.260281,3.281431,1.12436,163.3861,163.3861,14.57784,14.57784,10,0.0,0,0.0,0.0,0.0,0.0


# Compute regimes

In [18]:
grouped_sum = regime_data.group_by(["id", "branch"]).agg(pl.col("Harvested_V").sum().alias("sum_Harvested_V"))
threshold = grouped_sum.mean()["sum_Harvested_V"][0]

regime_data_w_sums = regime_data.join(grouped_sum.select(["id", "branch", "sum_Harvested_V"]), on=["id", "branch"], how="left")

result_column = (
    pl.when(pl.col("sum_Harvested_V") == 0)
    .then(pl.lit("SA"))
    .when(pl.col("sum_Harvested_V") <= threshold)
    .then(pl.lit("EXT"))
    .otherwise(pl.lit("INT"))
    .alias("branching_group")
)

regime_data = regime_data_w_sums.with_columns(result_column)

regime_data

id,branch,year,area,age,reg_age,reg_n,reg_n_S,reg_n_P,reg_n_B,reg_hDom,reg_hDom_S,reg_hDom_P,reg_hDom_B,reg_d,Tr,Rg,INC,COST,Net,Harvested_V,Harvested_V_S,Harvested_V_P,Harvested_V_B,Harvested_V_log,Harvested_V_log_S,Harvested_V_log_P,Harvested_V_log_B,Harvested_V_pulp,Harvested_V_pulp_S,Harvested_V_pulp_P,Harvested_V_pulp_B,n,n_S,n_P,n_B,h,…,h_mort.1,h_mort_spruce.1,h_mort_pine.1,h_mort_birch.1,d_mort.1,d_mort_spruce.1,d_mort_pine.1,d_mort_birch.1,ba_mort.1,ba_mort_spruce.1,ba_mort_pine.1,ba_mort_birch.1,biomass,soil,products,biomass.1,soil.1,products.1,NPV,Decay_class_1_volume,Decay_class_2_volume,Decay_class_3_volume,Decay_class_4_volume,Decay_class_5_volume,d_mort_pine_y,d_mort_spruce_y,vol_mort_pine_y,vol_mort_spruce_y,year.1,TTWP,CAP,HAZ,LSWP,SFS,LTT,sum_Harvested_V,branching_group
i64,i64,i64,f64,i64,i64,i64,f64,f64,i64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,f64,f64,f64,f64,f64,str
24473,1,1,2.03555,116,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,307.0,0.0,142.0,165.0,21.1219,…,0,0,0,0,0,0,0,0,0,0,0,0.877308,152.553,0.0,0.0,154.544,7.85294,0.0,43885.5,13.997524,0.0,0.0,0.0,0.0,43.6782,0.0,6.51497,0.0,1,0.791623,0,0.0,0.539321,0.0,0.84365,0.0,"""SA"""
25533,1,1,2.45563,0,1,170,0.0,170.0,0,1.35618,0.0,1.35618,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,777.112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0,0.0,0.0,0.0,0.0,0.0,"""SA"""
25533,1,2,2.45563,0,6,170,0.0,170.0,0,2.6482,0.0,2.6482,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,777.112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0,0.0,0.0,0.0,0.0,0.0,"""SA"""
24473,1,2,2.03555,121,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,296.106,0.0,136.961,159.145,21.5568,…,0,0,0,0,0,0,0,0,0,0,0,0.887889,154.544,7.85294,0.0,156.181,12.7199,0.0,43885.5,27.918172,0.0,0.0,0.0,0.0,88.42,0.0,13.22017,0.0,2,0.802918,0,0.0,0.565131,0.0,0.843649,0.0,"""SA"""
25533,1,3,2.45563,0,11,170,0.0,170.0,0,4.0775,0.0,4.0775,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,777.112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0,0.0,0.0,0.0,0.0,0.0,"""SA"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
61166,5,10,0.389153,61,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,19,63,142393.0,45009.0,97384.0,280.287,140.143,140.143,0.0,135.421,135.421,135.421,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0,0,0,0.0,98.2422,10.7366,0.0,0.0,26.0607,0.0,31725.6,14.841899,4.797826,3.260281,3.281431,1.12436,194.1876,194.1876,18.91532,18.91532,10,0.01143,0,0.0,0.015026,0.0,0.0,280.287,"""INT"""
61166,6,10,0.389153,0,5,3800,1910.0,90.0,1800,0.65,0.65,0.65,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,23.4304,0.0,0.0,15.8551,0.0,29261.1,6.522975,4.797826,3.260281,3.281431,1.12436,163.3861,163.3861,14.57784,14.57784,10,0.0,0,0.0,0.0,0.0,0.0,244.466,"""EXT"""
61166,7,10,0.389153,0,0,5200,1600.0,100.0,3500,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,23.4304,0.0,0.0,15.8551,0.0,32535.6,6.522975,4.797826,3.260281,3.281431,1.12436,163.3861,163.3861,14.57784,14.57784,10,0.0,0,0.0,0.0,0.0,0.0,244.466,"""EXT"""


# Combine the dataframes

In [61]:
combined = regime_data.join(
    cluster_data,
    left_on="id",
    right_on="BESTANDS_I",
    how="left")

combined

id,branch,year,area,age,reg_age,reg_n,reg_n_S,reg_n_P,reg_n_B,reg_hDom,reg_hDom_S,reg_hDom_P,reg_hDom_B,reg_d,Tr,Rg,INC,COST,Net,Harvested_V,Harvested_V_S,Harvested_V_P,Harvested_V_B,Harvested_V_log,Harvested_V_log_S,Harvested_V_log_P,Harvested_V_log_B,Harvested_V_pulp,Harvested_V_pulp_S,Harvested_V_pulp_P,Harvested_V_pulp_B,n,n_S,n_P,n_B,h,…,h_mort_birch.1,d_mort.1,d_mort_spruce.1,d_mort_pine.1,d_mort_birch.1,ba_mort.1,ba_mort_spruce.1,ba_mort_pine.1,ba_mort_birch.1,biomass,soil,products,biomass.1,soil.1,products.1,NPV,Decay_class_1_volume,Decay_class_2_volume,Decay_class_3_volume,Decay_class_4_volume,Decay_class_5_volume,d_mort_pine_y,d_mort_spruce_y,vol_mort_pine_y,vol_mort_spruce_y,year.1,TTWP,CAP,HAZ,LSWP,SFS,LTT,,_duplicated_0,_duplicated_1,Forest_own,geometry
i64,i64,i64,f64,i64,i64,i64,f64,f64,i64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,…,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,f64,f64,f64,f64,str,f64,str,i64,str
24473,1,1,2.03555,116,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,307.0,0.0,142.0,165.0,21.1219,…,0,0,0,0,0,0,0,0,0.877308,152.553,0.0,0.0,154.544,7.85294,0.0,43885.5,13.997524,0.0,0.0,0.0,0.0,43.6782,0.0,6.51497,0.0,1,0.791623,0,0.0,0.539321,0.0,0.84365,null,null,null,211,"""POLYGON ((608511.693783 659739…"
25533,1,1,2.45563,0,1,170,0.0,170.0,0,1.35618,0.0,1.35618,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,777.112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0,0.0,0.0,0.0,0.0,null,null,null,114,"""POLYGON ((607326.931065 659950…"
25533,1,2,2.45563,0,6,170,0.0,170.0,0,2.6482,0.0,2.6482,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,777.112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0,0.0,0.0,0.0,0.0,null,null,null,114,"""POLYGON ((607326.931065 659950…"
24473,1,2,2.03555,121,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,296.106,0.0,136.961,159.145,21.5568,…,0,0,0,0,0,0,0,0,0.887889,154.544,7.85294,0.0,156.181,12.7199,0.0,43885.5,27.918172,0.0,0.0,0.0,0.0,88.42,0.0,13.22017,0.0,2,0.802918,0,0.0,0.565131,0.0,0.843649,null,null,null,211,"""POLYGON ((608511.693783 659739…"
25533,1,3,2.45563,0,11,170,0.0,170.0,0,4.0775,0.0,4.0775,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,777.112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0,0.0,0.0,0.0,0.0,null,null,null,114,"""POLYGON ((607326.931065 659950…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
61166,5,10,0.389153,61,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,19,63,142393.0,45009.0,97384.0,280.287,140.143,140.143,0.0,135.421,135.421,135.421,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0.0,98.2422,10.7366,0.0,0.0,26.0607,0.0,31725.6,14.841899,4.797826,3.260281,3.281431,1.12436,194.1876,194.1876,18.91532,18.91532,10,0.01143,0,0.0,0.015026,0.0,0.0,null,null,null,222,"""POLYGON ((607587.283292 659910…"
61166,6,10,0.389153,0,5,3800,1910.0,90.0,1800,0.65,0.65,0.65,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0.0,0.0,23.4304,0.0,0.0,15.8551,0.0,29261.1,6.522975,4.797826,3.260281,3.281431,1.12436,163.3861,163.3861,14.57784,14.57784,10,0.0,0,0.0,0.0,0.0,0.0,null,null,null,222,"""POLYGON ((607587.283292 659910…"
61166,7,10,0.389153,0,0,5200,1600.0,100.0,3500,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,…,0,0,0,0,0,0,0,0,0.0,0.0,23.4304,0.0,0.0,15.8551,0.0,32535.6,6.522975,4.797826,3.260281,3.281431,1.12436,163.3861,163.3861,14.57784,14.57784,10,0.0,0,0.0,0.0,0.0,0.0,null,null,null,222,"""POLYGON ((607587.283292 659910…"